# Programação Paralela Híbrida: MPI + OpenMP Offloading

Autores: *Calebe P. Bianchini, Evaldo B. Costa, Gabriel P. Silva*

### Setup rápido do ambiente

(não esqueça de selecionar um Runtime adequado, com GPU)


In [1]:
!add-apt-repository -y ppa:ubuntu-toolchain-r/test &> /dev/null
!apt install -y gcc-13 g++-13 gcc-13-offload-nvptx libgomp1 &> /dev/null
!apt install openmpi-bin openmpi-common libopenmpi-dev &> /dev/null
!ln -sfnv /usr/bin/gcc-13 /usr/bin/gcc &> /dev/null
!gcc --version
!mpicc --showme:version
!nvcc --version
!nvidia-smi

gcc (Ubuntu 13.1.0-8ubuntu1~22.04) 13.1.0
Copyright (C) 2023 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

mpicc: Open MPI 4.1.2 (Language: C)
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Fri Oct 25 05:38:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                            

## Problema do PI

1. Criação do arquivo de código-fonte

In [43]:
%%writefile pi.c

#include <mpi.h>
#include <stdio.h>
#include <omp.h>
#include <math.h>

int main(int argc, char *argv[]) {
  MPI_Init(&argc, &argv);
  int rank, size;
  long int num_steps = 10000000000; // Número de passos para a integração
  double step = 1.0 / (double) num_steps;
  double pi = 0.0;
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &size);
  if(size < 2) {
    MPI_Abort(MPI_COMM_WORLD, -1);
  }
  long int my_num_steps = num_steps / size;
  long int i_begin = rank * my_num_steps;
  long int i_end = i_begin + my_num_steps;
  // Versão encadeada de execução na GPU, controlada por mensagens MPI a partir do rank 0
  if(!rank) {
    double begin, end;
    printf("Iniciando o calculo do Pi no rank: %d\n", rank);
    begin = omp_get_wtime();
  #pragma omp target data map(tofrom: pi) map(to:num_steps, step) device(1)// Diretiva para offloading para a GPU
  #pragma omp target teams distribute parallel for reduction(+:pi) // Paralelização com OpenMP no rank 0
    for (long int i = i_begin; i < i_end; i++) {
          double x = (i + 0.5) * step;
          pi += 4.0 / (double) (1.0 + x * x);
    }
    printf("Valor parcial do Pi: %2.15f\n", pi*step);
    MPI_Send(&pi, 1, MPI_DOUBLE, 1, num_steps, MPI_COMM_WORLD);
    MPI_Recv(&pi, 1, MPI_DOUBLE, MPI_ANY_SOURCE, MPI_ANY_TAG, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    pi *= step;
    end = omp_get_wtime();
    printf("Valor de Pi calculado: %2.15f\n", pi);
    printf("Tempo de execução: %f segundos\n", end - begin);
  } else {
    MPI_Recv(&pi, 1, MPI_DOUBLE, rank-1, MPI_ANY_TAG, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    printf("Continuando o calculo do Pi no rank: %d\n", rank);
  #pragma omp target data map(tofrom: pi) map(to:num_steps, step) device(1)// Diretiva para offloading para a GPU
  #pragma omp target teams distribute parallel for reduction(+:pi) // Paralelização com OpenMP nos demais ranks
     for (long int i = i_begin; i < i_end; i++) {
          double x = (i + 0.5) * step;
          pi += 4.0 / (double) (1.0 + x * x);
     }
     printf("Valor parcial do Pi: %2.15f\n", pi*step);
     MPI_Send(&pi, 1, MPI_DOUBLE, (rank+1)%size, num_steps, MPI_COMM_WORLD);
  }
  MPI_Finalize();
  return 0;
}

Overwriting pi.c


2. Compilação com MPI e OpenMP _offload_

In [44]:
!mpicc -fopenmp -fno-lto -fstack-protector pi.c -Wall -o pi

3. Executando o cálculo do Pi com OpenMP Offloading e de forma cadenciada controlada por mensagens MPI

In [45]:
!mpirun --allow-run-as-root --host localhost:4 -np 4 ./pi

Iniciando o calculo do Pi no rank: 0
Valor parcial do Pi: 0.979914652507362
Continuando o calculo do Pi no rank: 1
Valor parcial do Pi: 1.854590436003130
Continuando o calculo do Pi no rank: 2
Valor parcial do Pi: 2.574004435173032
Continuando o calculo do Pi no rank: 3
Valor parcial do Pi: 3.141592653589691
Valor de Pi calculado: 3.141592653589691
Tempo de execução: 73.058189 segundos
